# Postprocesado de segmentaciones vertebrales
En este cuaderno, se implementa el postprocesado de las segmentaciones obtenidas del modelo aplicado a las imágenes de radiografías de columna. Se aplican distintas etapas:
* Refinado morfológico
* Filtrado por área
* Separación de regiones grandes
* Agrupamiento con DBSCAN
* Analisis de estructura
* Selección del mejor candidato a ser columna (entre los conjuntos obtenidos)

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

from scipy import ndimage as ndi
from skimage.feature import peak_local_max
from skimage.segmentation import watershed
from skimage.segmentation import find_boundaries

from sklearn.cluster import DBSCAN

Se cargan las rutas y parametros

In [ ]:
ruta_procesado = 'dataset_procesado'

con_mascara = 'dataset_con_mascara'
sin_mascara = 'dataset_sin_mascara'

mod_cervical = 'cervical'
mod_lumbar = 'lumbar'
mod_general = 'general'

imagenes = 'imagenes'
mascaras = 'mascaras'
mascara_base = 'segmentaciones'
mascara_limpias = 'segmentaciones_limpias'

In [ ]:
ROJO = (255, 0, 0)
VERDE = (0, 255, 0)
AZUL = (0, 0, 255)
NARANJA = (255, 165, 0)

In [ ]:
kernel_3 = np.ones((3, 3), np.uint8)
kernel_5 = np.ones((5, 5), np.uint8)
kernel_7 = np.ones((7, 7), np.uint8)

## Funciones auxiliares

In [ ]:
# Actualiza la máscara eliminando contornos no deseados
def borrar_contornos(mascara, contornos_descartados):
  mascara_actualizada = mascara.copy()
  for c in contornos_descartados:
    cv2.drawContours(mascara_actualizada, [c], -1, 0, thickness=cv2.FILLED)
  return mascara_actualizada

# Actualiza la máscara dibujando contornos deseados
def actualizar_mascara(mascara, contornos):
  mascara_actualizada = mascara.copy()
  mascara_actualizada = np.zeros_like(mascara)
  for c in contornos:
    cv2.drawContours(mascara_actualizada, [c], -1, 1, -1)
  return mascara_actualizada

In [ ]:
def calcular_centroides(contornos):
  centroides = []
  for c in contornos:
    if len(c) > 5:
      elipse = cv2.fitEllipse(c)
      centro = elipse[0]
      centro = (int(centro[0]), int(centro[1]))
      centroides.append(centro)
  return centroides

In [ ]:
def calcular_rango_areas(ruta_mascaras):
  min_area_cervical, max_area_cervical = float('inf'), float('-inf')
  min_area_c1, max_area_c1 = float('inf'), float('-inf')
  min_area_lumbar, max_area_lumbar = float('inf'), float('-inf')

  for nombre_img in os.listdir(ruta_mascaras):
    ruta = os.path.join(ruta_mascaras, nombre_img)
    mascara = cv2.imread(ruta, cv2.IMREAD_GRAYSCALE)
    if mascara is None:
      continue
    contornos, _ = cv2.findContours(mascara, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    areas = [cv2.contourArea(c) for c in contornos]
    if not areas:
      continue

    if 'cervical' in nombre_img.lower():
      area_c1 = areas[-1]
      otras = areas[:-1] if len(areas) > 1 else []
      min_area_c1 = min(min_area_c1, area_c1)
      max_area_c1 = max(max_area_c1, area_c1)
      if otras:
        min_area_cervical = min(min_area_cervical, min(otras))
        max_area_cervical = max(max_area_cervical, max(otras))
    elif 'lumbar' in nombre_img.lower():
      min_area_lumbar = min(min_area_lumbar, min(areas))
      max_area_lumbar = max(max_area_lumbar, max(areas))

  media_cevical = (min_area_cervical + max_area_cervical) / 2
  media_c1 = (min_area_c1 + max_area_c1) / 2
  media_lumbar = (min_area_lumbar + max_area_lumbar) / 2

  area_cervical = [[min_area_cervical, media_cevical, max_area_cervical],
                   [min_area_c1, media_c1, max_area_c1]]
  area_lumbar = [min_area_lumbar, media_lumbar, max_area_lumbar]

  print(f"Area cervical: {area_cervical}")
  print(f"Area lumbar: {area_lumbar}")
  return area_cervical, area_lumbar

mascaras_reales = os.path.join(ruta_procesado, con_mascara, mod_general, mascaras)
AREA_CERVICAL, AREA_LUMBAR = calcular_rango_areas(mascaras_reales)

### Funciones de refinado morfológico

In [ ]:
def refinado_morfonologico_cervical(mask, kernel, iteraciones):
  mask = cv2.erode(mask, kernel, iterations=iteraciones+2)
  mask = cv2.dilate(mask, kernel, iterations=iteraciones)
  return mask

def refinado_morfonologico_lumbar(mask, kernel, iteraciones):
  mask = cv2.erode(mask, kernel, iterations=iteraciones+2)
  mask = cv2.dilate(mask, kernel, iterations=iteraciones)
  return mask

### Funciones para filtrado y separación de vértebras
#### Filtrado de areas pequeñas

In [ ]:
def filtrar_areas_pequenas(contornos, areas, tipo='cervical'):
  umbral = AREA_CERVICAL[0][0] if tipo == 'cervical' else AREA_LUMBAR[0]
  contornos_filtrados = []
  for i, c in enumerate(contornos):
    #print(f"Area: {areas[i]}")
    if areas[i] > umbral:
      contornos_filtrados.append(c)

  return contornos_filtrados

#### Separación de regiones grandes

In [ ]:
def separar_vertebras_con_watershed(region_mask, min_distance=10, debug=False):
    region_mask = (region_mask > 0).astype(np.uint8)
    # Suavizado para eliminar irregularidades
    kernel = np.ones((3, 3), np.uint8)
    region_mask = cv2.morphologyEx(region_mask, cv2.MORPH_CLOSE, kernel)
    region_mask = cv2.morphologyEx(region_mask, cv2.MORPH_OPEN, kernel)
    # Transformada de distancia
    distance = ndi.distance_transform_edt(region_mask) # distancia de cada píxel al borde más cercano
    
    # Picos locales bien espaciados
    coordinates = peak_local_max(distance, min_distance=min_distance, labels=region_mask)
    local_max = np.zeros_like(region_mask, dtype=bool)
    for c in coordinates:
        local_max[c[0], c[1]] = True
    if np.sum(local_max) == 0:
        print("No se encontraron picos. Saltando watershed.")
        return []

    markers = ndi.label(local_max)[0]
    labels = watershed(-distance, markers, mask=region_mask)

    nuevos_contornos = []
    for etiqueta in np.unique(labels):
        if etiqueta == 0:
            continue
        componente = (labels == etiqueta).astype(np.uint8)
        contornos, _ = cv2.findContours(componente, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        nuevos_contornos.extend(contornos)

    # Delimitar los contornos encontrados
    boundary = find_boundaries(labels, mode='thick')
    boundary_dilated = cv2.dilate(boundary.astype(np.uint8), kernel, iterations=2)
    region_mask_separada = region_mask.copy()
    region_mask_separada[boundary_dilated>0] = 0

    num_componentes, componente_labels = cv2.connectedComponents(region_mask_separada.astype(np.uint8))
    contornos_separados = []
    for val in range(1, num_componentes):  # Saltamos el fondo (label 0)
        componente = (componente_labels == val).astype(np.uint8)
        contornos, _ = cv2.findContours(componente, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contornos_separados.extend(contornos)

    if debug:
        plt.figure(figsize=(8, 4))
        plt.subplot(1, 3, 1)
        plt.imshow(region_mask, cmap='gray')
        plt.title("Máscara suavizada")
        plt.subplot(1, 3, 2)
        plt.imshow(distance, cmap='magma')
        plt.title("Transformada de distancia")
        plt.subplot(1, 3, 3)
        plt.imshow(labels, cmap='nipy_spectral')
        plt.title("Segmentación (Watershed)")
        plt.tight_layout()
        plt.show()

    return contornos_separados

In [ ]:
def separar_vertebras(mascara, contornos, umbral, min_distance=10, debug=False): 
  areas = [cv2.contourArea(c) for c in contornos]
  contornos_divididos = []
  
  for c, a in zip(contornos, areas):
    if a > umbral:
      mask_temp = np.zeros_like(mascara)
      cv2.drawContours(mask_temp, [c], -1, 1, -1)
      nuevos = separar_vertebras_con_watershed(mask_temp, min_distance, debug)
      contornos_divididos.extend(nuevos)
    else:
      contornos_divididos.append(c)
  
  return contornos_divididos

### Funciones de clustering y análisis
#### Agrupamiento con DBSCAN

In [ ]:
def filtrar_con_clustering(contornos, tipo):
  centroides = calcular_centroides(contornos)
  # Elimina outliers con clustering
  if len(centroides) > 0:
    centroides = np.array(centroides)
    # DBSCAN
    umbral = 250 if tipo == 'cervical' else 400
    db = DBSCAN(eps=umbral, min_samples=2).fit(centroides)
    etiquetas = db.labels_

  contornos_filtrados = [c for c, e in zip(contornos, etiquetas) if e != -1]
  etiquetas = etiquetas[etiquetas != -1]

  return contornos_filtrados, etiquetas

#### Analisis de estructura

In [ ]:
def calcular_direccion(centroides):
  if len(centroides) < 2:
    return None

  cov = np.cov(centroides.T)
  valores, vectores = np.linalg.eig(cov)
  direccion_principal = vectores[:, np.argmax(valores)]

  return direccion_principal

In [ ]:
def filtrar_centroides_desalineados(centroides, umbral=0.90, debug=False):
  if len(centroides) < 3:
    return centroides
  
  centroides = np.array(centroides)
  centroide_medio = np.mean(centroides, axis=0)
  centroides_centrados = centroides - centroide_medio
  
  direccion_principal = calcular_direccion(centroides_centrados)

  proyeccion = centroides_centrados @ direccion_principal
  orden = np.argsort(proyeccion)

  mejor_columna = []
  mejor_puntuacion = -1

  for inicio in range(len(centroides) - 1):
    indices = [orden[inicio], orden[inicio + 1]]
    direccion_acumulada = centroides[indices[1]] - centroides[indices[0]]
    direccion_acumulada = direccion_acumulada / np.linalg.norm(direccion_acumulada)

    for i in range(inicio + 2, len(orden)):
      idx_actual = orden[i]
      vector = centroides[idx_actual] - centroides[indices[-1]]
      norma = np.linalg.norm(vector)
      if norma == 0:
        continue
      vector = vector / norma
      coseno = np.dot(direccion_acumulada, vector)
      if coseno >= umbral:
        indices.append(idx_actual)
        direccion_acumulada = (direccion_acumulada + vector) / 2
        direccion_acumulada = direccion_acumulada / np.linalg.norm(direccion_acumulada)
      elif debug:
        print(f"Desalineado: {centroides[idx_actual]} con coseno {coseno:.2f}")

    if len(indices) > mejor_puntuacion:
      mejor_columna = indices.copy()
      mejor_puntuacion = len(indices)
  return [tuple(centroides[i]) for i in mejor_columna]

In [ ]:
def es_posible_columna(centroides, min_vertebras=3, max_std_dist=70, debug=False):
  if len(centroides) < 2:
    return False, 0

  centroides = np.array(centroides)
  centroide_medio = np.mean(centroides, axis=0)
  centroides_centrados = centroides - centroide_medio

  direccion_principal = calcular_direccion(centroides_centrados)

  proyeccion = centroides_centrados @ direccion_principal
  orden = np.argsort(proyeccion)
  centroides_ordenados = centroides[orden]

  # Distancia entre centroides consecutivos
  vectores = np.diff(centroides_ordenados, axis=0)
  distancias = np.linalg.norm(vectores, axis=1)
  std_dist = np.std(distancias)
  n_vertebras = len(centroides_ordenados)

  if debug:
    print(f"Distancias: {distancias}")
    print(f"Desviación distancias: {std_dist}")
    print(f"Dirección principal: {direccion_principal}")
    print("------------------------------------------------")

  es_valida = (n_vertebras >= min_vertebras and std_dist < max_std_dist)
  puntuacion = (n_vertebras ** 2) - std_dist

  return es_valida, puntuacion

#### Selección del mejor candidato a ser columna (entre los conjuntos obtenidos)

In [ ]:
def seleccionar_columna(contornos, centroides, etiquetas):
  print(f"Seleccionar columna: {len(contornos)} contornos iniciales")
  if len(contornos) >= 3:
    grupos = {}
    for i, etiqueta in enumerate(etiquetas):
      if etiqueta not in grupos:
        grupos[etiqueta] = {'contornos': [], 'centroides': []}
      grupos[etiqueta]['contornos'].append(contornos[i])
      grupos[etiqueta]['centroides'].append(centroides[i])

    mejor_grupo = None
    mejor_puntuacion = -np.inf
    puntuaciones_por_grupo = []

    for etiqueta, datos in grupos.items():
      centroides_limpios = filtrar_centroides_desalineados(datos['centroides'])
      es_posible, puntuacion = es_posible_columna(centroides_limpios)
      if len(centroides_limpios) > 2:
        puntuaciones_por_grupo.append((etiqueta, centroides_limpios, puntuacion, es_posible))

      if es_posible and puntuacion > mejor_puntuacion:
        mejor_puntuacion = puntuacion
        mejor_grupo = (etiqueta, centroides_limpios)
        
    # Si ningún grupo pasó el filtro, quedarnos con el de mayor puntuación
    if mejor_grupo is None and puntuaciones_por_grupo:
      puntuaciones_por_grupo.sort(key=lambda x: x[2], reverse=True)
      etiqueta, centroides_limpios, _, _ = puntuaciones_por_grupo[0]
      mejor_grupo = (etiqueta, centroides_limpios)

    if mejor_grupo:
      etiqueta_valida, centroides_validos = mejor_grupo
      print(f"Etiqueta válida: {etiqueta_valida}, Centroides válidos: {centroides_validos}")
      print(f"centroides: {centroides}")
      contornos_filtados = [
        contornos[i] for i in range(len(contornos))
        if etiquetas[i] == etiqueta_valida and centroides[i] in centroides_validos
      ]

      print(f" → Devueltos: {len(contornos_filtados)} contornos, {len(centroides_validos)} centroides")
      return contornos_filtados, centroides_validos
  return [], []

## Código principal del postprocesado

In [ ]:
def procesar_mascara(mascara, tipo, imagen, debug=False):
  if debug:
    print(f"Procesando imagen con tipo: {tipo}")
    plt.imshow(mascara, cmap='gray')
    plt.title(f"Máscara original")
    plt.show()

  procesado = []
  # Refinado morfológico
  if tipo == 'cervical':
    procesado = refinado_morfonologico_cervical(mascara, kernel_3, 6)
  elif tipo == 'lumbar':
    procesado = refinado_morfonologico_lumbar(mascara, kernel_3, 3)
  else:
    print(f"Modelo no reconocido para la imagen")
    return None
  
  if debug:
    plt.imshow(procesado, cmap='gray')
    plt.title(f"Refinado morfológico")
    plt.show()
  
  # Filtrar regiones pequeñas
  contornos, _ = cv2.findContours(procesado, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  areas = [cv2.contourArea(c) for c in contornos]

  regiones_grandes = filtrar_areas_pequenas(contornos, areas, tipo)
  regiones_descartadas = [c for c in contornos if all(not np.array_equal(c, ca) for ca in regiones_grandes)]
  mascara_actualizada = borrar_contornos(procesado, regiones_descartadas)

  if debug:
    plt.imshow(mascara_actualizada, cmap='gray')
    plt.title(f"Sin regiones pequeñas")
    plt.show()

  # Separar regiones grandes
  contornos = regiones_grandes
  umbral_superior = AREA_CERVICAL[0][2] if tipo == 'cervical' else AREA_LUMBAR[2]
  contornos = separar_vertebras(mascara_actualizada, contornos, umbral_superior, 30, debug)
  areas = [cv2.contourArea(c) for c in contornos]

  # Elimina contornos pequeños
  contornos_filtrados = filtrar_areas_pequenas(contornos, areas, tipo)
  mascara_actualizada = actualizar_mascara(mascara, contornos_filtrados)
  contornos, _ = cv2.findContours(mascara_actualizada, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  areas = [cv2.contourArea(c) for c in contornos]

  if debug:
    plt.imshow(mascara_actualizada, cmap='gray')
    plt.title(f"Regiones separadas")
    plt.show()

  # Elimina outliers con clustering
  contornos, etiquetas = filtrar_con_clustering(contornos, tipo)
  mascara_actualizada = actualizar_mascara(mascara_actualizada, contornos)
  areas = [cv2.contourArea(c) for c in contornos]

  if debug:
    print(f"Etiquetas encontradas: {set(etiquetas)}")
    plt.imshow(mascara_actualizada, cmap='gray')
    plt.title(f"DBSCAN")
    plt.show()

  if tipo == 'cervical':
    if len(contornos) > 0:
      centroides = calcular_centroides(contornos)
      #if len(set(etiquetas)) > 1:
      contornos_filtrados, centroides_filtrados = seleccionar_columna(contornos, centroides, etiquetas)
      mascara_actualizada = actualizar_mascara(mascara_actualizada, contornos_filtrados)
      plt.imshow(mascara_actualizada, cmap='gray')
      plt.title(f"Columna cervical")
      plt.show()
      #else:
      #  centroides_filtrados = filtrar_centroides_desalineados(centroides)

      return mascara_actualizada
  else:
    return mascara_actualizada

## Aplicación del postprocesado
### Sobre modelos general
Se aplica a las segmentaciones obtenidas por el modelo general
1. Sobre el conjuntos NHANESS II

In [ ]:
# Limpieza de segmentaciones para el modelo general
ruta_entrada = os.path.join(ruta_procesado, con_mascara, mod_general)
ruta_salida = os.path.join(ruta_entrada, mascara_limpias)
os.makedirs(ruta_salida, exist_ok=True)

for img_name in os.listdir(os.path.join(ruta_entrada, imagenes)):
  print(f"Procesando imagen: {img_name}")
  if not img_name.endswith('.png'):
    continue
  ruta_imagen = os.path.join(ruta_entrada, imagenes, img_name)
  ruta_mascara = os.path.join(ruta_entrada, mascaras, img_name)
  ruta_prediccion = os.path.join(ruta_entrada, mascara_base, img_name)

  imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
  mascara = cv2.imread(ruta_mascara, cv2.IMREAD_GRAYSCALE)
  prediccion = cv2.imread(ruta_prediccion, cv2.IMREAD_GRAYSCALE)

  radio1 = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
  radio1 = cv2.cvtColor(radio1, cv2.COLOR_GRAY2BGR)
  radio2 = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
  radio2 = cv2.cvtColor(radio2, cv2.COLOR_GRAY2BGR)

  contornos_pr, _ = cv2.findContours(prediccion, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cv2.drawContours(radio1, contornos_pr, -1, AZUL, cv2.FILLED)

  tipo = 'cervical' if 'cervical' in img_name.lower() else 'lumbar'
  segmentacion_limpia = procesar_mascara(prediccion, tipo, img_name, debug=False)
  if segmentacion_limpia is None:
    print(f"Error al procesar la máscara: {img_name}")
    continue

  contornos, _ = cv2.findContours(segmentacion_limpia, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cv2.drawContours(radio2, contornos, -1, AZUL, cv2.FILLED)

  if segmentacion_limpia is not None:
    cv2.imwrite(os.path.join(ruta_salida, img_name), segmentacion_limpia)

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 3, 1)
    plt.imshow(mascara, cmap='gray')
    plt.title("Mascara original")
    plt.subplot(1, 3, 2)
    plt.imshow(radio1, cmap='gray')
    plt.title("Segmentacion obtenida")
    plt.subplot(1, 3, 3)
    plt.imshow(radio2, cmap='gray')
    plt.title("Segmentacion postprocesada")
    plt.tight_layout()

    # guardar figura en resultados_post_procesado
    ruta_resultados = os.path.join('resultados_post_procesado', 'general')
    os.makedirs(ruta_resultados, exist_ok=True)
    ruta_figura = os.path.join(ruta_resultados, f"{img_name}.png")
    plt.savefig(ruta_figura, bbox_inches='tight', dpi=300)

    plt.show()


2. Sobre el conjunto proporcionado por el médico

In [ ]:
# Limpieza de segmentaciones para el modelo general
ruta_entrada = os.path.join(ruta_procesado, sin_mascara, mod_general)
ruta_salida = os.path.join(ruta_entrada, mascara_limpias)
os.makedirs(ruta_salida, exist_ok=True)

for img_name in os.listdir(os.path.join(ruta_entrada, imagenes)):
  if not img_name.endswith('.png'):
    continue
  print(f"Procesando imagen: {img_name}")
  ruta_imagen = os.path.join(ruta_entrada, imagenes, img_name)
  ruta_prediccion = os.path.join(ruta_entrada, mascara_base, img_name)

  imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
  prediccion = cv2.imread(ruta_prediccion, cv2.IMREAD_GRAYSCALE)

  tipo = 'cervical' if 'cervical' in img_name.lower() else 'lumbar'
  segmentacion_limpia = procesar_mascara(prediccion, tipo,imagen)
  if segmentacion_limpia is not None:
    cv2.imwrite(os.path.join(ruta_salida, img_name), segmentacion_limpia)
    
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(prediccion, cmap='gray')
    plt.title("Segmentacion original")
    plt.subplot(1, 2, 2)
    plt.imshow(segmentacion_limpia, cmap='gray')
    plt.title("Segmentacion postprocesada")
    plt.tight_layout()

    # guardar figura en resultados_post_procesado
    ruta_resultados = os.path.join('resultados_post_procesado_julio', 'general')
    os.makedirs(ruta_resultados, exist_ok=True)
    ruta_figura = os.path.join(ruta_resultados, f"{img_name}.png")
    plt.savefig(ruta_figura, bbox_inches='tight', dpi=300)

    plt.show()
  else:
    print(f"Segmentación nula para la imagen: {img_name}")
    plt.imshow(prediccion, cmap='gray')
    plt.title("Segmentacion original")
    plt.show()
    

### Sobre los modelos de la región cervical
Se aplica a las segmentaciones obtenidas por el modelo entrenado con imágenes de la región cervical
1. Sobre el conjuntos NHANESS II

In [ ]:
# Limpieza de segmentaciones para el modelo cervical
ruta_entrada = os.path.join(ruta_procesado, con_mascara, mod_cervical)
ruta_salida = os.path.join(ruta_entrada, mascara_limpias)
os.makedirs(ruta_salida, exist_ok=True)

for img_name in os.listdir(os.path.join(ruta_entrada, imagenes)):
  print(f"Procesando imagen: {img_name}")
  if not img_name.endswith('.png'):
    continue
  ruta_imagen = os.path.join(ruta_entrada, imagenes, img_name)
  ruta_mascara = os.path.join(ruta_entrada, mascaras, img_name)
  ruta_prediccion = os.path.join(ruta_entrada, mascara_base, img_name)

  imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
  mascara = cv2.imread(ruta_mascara, cv2.IMREAD_GRAYSCALE)
  prediccion = cv2.imread(ruta_prediccion, cv2.IMREAD_GRAYSCALE)

  radio1 = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
  radio1 = cv2.cvtColor(radio1, cv2.COLOR_GRAY2BGR)
  radio2 = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
  radio2 = cv2.cvtColor(radio2, cv2.COLOR_GRAY2BGR)

  contornos_pr, _ = cv2.findContours(prediccion, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cv2.drawContours(radio1, contornos_pr, -1, AZUL, cv2.FILLED)

  tipo = 'cervical' if 'cervical' in img_name.lower() else 'lumbar'
  segmentacion_limpia = procesar_mascara(prediccion, tipo, img_name, debug=False)
  if segmentacion_limpia is None:
    print(f"Error al procesar la máscara: {img_name}")
    continue

  contornos, _ = cv2.findContours(segmentacion_limpia, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cv2.drawContours(radio2, contornos, -1, AZUL, cv2.FILLED)

  if segmentacion_limpia is not None:
    cv2.imwrite(os.path.join(ruta_salida, img_name), segmentacion_limpia)

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 3, 1)
    plt.imshow(mascara, cmap='gray')
    plt.title("Mascara original")
    plt.subplot(1, 3, 2)
    plt.imshow(radio1, cmap='gray')
    plt.title("Segmentacion obtenida")
    plt.subplot(1, 3, 3)
    plt.imshow(radio2, cmap='gray')
    plt.title("Segmentacion postprocesada")
    plt.tight_layout()

    # guardar figura en resultados_post_procesado
    ruta_resultados = os.path.join('resultados_post_procesado', 'cervical')
    os.makedirs(ruta_resultados, exist_ok=True)
    ruta_figura = os.path.join(ruta_resultados, f"{img_name}.png")
    plt.savefig(ruta_figura, bbox_inches='tight', dpi=300)

    plt.show()

2. Sobre el conjunto proporcionado por el médico

In [ ]:
# Limpieza de segmentaciones para el modelo cervical
ruta_entrada = os.path.join(ruta_procesado, sin_mascara, mod_cervical)
ruta_salida = os.path.join(ruta_entrada, mascara_limpias)
os.makedirs(ruta_salida, exist_ok=True)

for img_name in os.listdir(os.path.join(ruta_entrada, imagenes)):
  if not img_name.endswith('.png'):
    continue
  print(f"Procesando imagen: {img_name}")
  ruta_imagen = os.path.join(ruta_entrada, imagenes, img_name)
  ruta_prediccion = os.path.join(ruta_entrada, mascara_base, img_name)

  imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
  prediccion = cv2.imread(ruta_prediccion, cv2.IMREAD_GRAYSCALE)

  tipo = 'cervical' if 'cervical' in img_name.lower() else 'lumbar'
  segmentacion_limpia = procesar_mascara(prediccion, tipo,imagen)
  if segmentacion_limpia is not None:
    cv2.imwrite(os.path.join(ruta_salida, img_name), segmentacion_limpia)
    
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(prediccion, cmap='gray')
    plt.title("Segmentacion original")
    plt.subplot(1, 2, 2)
    plt.imshow(segmentacion_limpia, cmap='gray')
    plt.title("Segmentacion postprocesada")
    plt.tight_layout()

    # guardar figura en resultados_post_procesado
    ruta_resultados = os.path.join('resultados_post_procesado_julio', 'cervical')
    os.makedirs(ruta_resultados, exist_ok=True)
    ruta_figura = os.path.join(ruta_resultados, f"{img_name}.png")
    plt.savefig(ruta_figura, bbox_inches='tight', dpi=300)

    plt.show()
  else:
    print(f"Segmentación nula para la imagen: {img_name}")
    plt.imshow(prediccion, cmap='gray')
    plt.title("Segmentacion original")
    plt.show()
    

### Sobre los modelos de la región lumbar
Se aplica a las segmentaciones obtenidas por el modelo entrenado con imágenes de la región lumbar
1. Sobre el conjuntos NHANESS II

In [ ]:
# Limpieza de segmentaciones para el modelo lumbar
ruta_entrada = os.path.join(ruta_procesado, con_mascara, mod_lumbar)
ruta_salida = os.path.join(ruta_entrada, mascara_limpias)
os.makedirs(ruta_salida, exist_ok=True)

for img_name in os.listdir(os.path.join(ruta_entrada, imagenes)):
  print(f"Procesando imagen: {img_name}")
  if not img_name.endswith('.png'):
    continue
  ruta_imagen = os.path.join(ruta_entrada, imagenes, img_name)
  ruta_mascara = os.path.join(ruta_entrada, mascaras, img_name)
  ruta_prediccion = os.path.join(ruta_entrada, mascara_base, img_name)

  imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
  mascara = cv2.imread(ruta_mascara, cv2.IMREAD_GRAYSCALE)
  prediccion = cv2.imread(ruta_prediccion, cv2.IMREAD_GRAYSCALE)

  radio1 = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
  radio1 = cv2.cvtColor(radio1, cv2.COLOR_GRAY2BGR)
  radio2 = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
  radio2 = cv2.cvtColor(radio2, cv2.COLOR_GRAY2BGR)

  contornos_pr, _ = cv2.findContours(prediccion, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cv2.drawContours(radio1, contornos_pr, -1, AZUL, cv2.FILLED)

  tipo = 'cervical' if 'cervical' in img_name.lower() else 'lumbar'
  segmentacion_limpia = procesar_mascara(prediccion, tipo, img_name, debug=False)
  if segmentacion_limpia is None:
    print(f"Error al procesar la máscara: {img_name}")
    continue

  contornos, _ = cv2.findContours(segmentacion_limpia, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cv2.drawContours(radio2, contornos, -1, AZUL, cv2.FILLED)

  if segmentacion_limpia is not None:
    cv2.imwrite(os.path.join(ruta_salida, img_name), segmentacion_limpia)

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 3, 1)
    plt.imshow(mascara, cmap='gray')
    plt.title("Mascara original")
    plt.subplot(1, 3, 2)
    plt.imshow(radio1, cmap='gray')
    plt.title("Segmentacion obtenida")
    plt.subplot(1, 3, 3)
    plt.imshow(radio2, cmap='gray')
    plt.title("Segmentacion postprocesada")
    plt.tight_layout()

    # guardar figura en resultados_post_procesado
    ruta_resultados = os.path.join('resultados_post_procesado', 'lumbar')
    os.makedirs(ruta_resultados, exist_ok=True)
    ruta_figura = os.path.join(ruta_resultados, f"{img_name}.png")
    plt.savefig(ruta_figura, bbox_inches='tight', dpi=300)

    plt.show()

2. Sobre el conjunto proporcionado por el médico

In [ ]:
# Limpieza de segmentaciones para el modelo lumbar
ruta_entrada = os.path.join(ruta_procesado, sin_mascara, mod_lumbar)
ruta_salida = os.path.join(ruta_entrada, mascara_limpias)
os.makedirs(ruta_salida, exist_ok=True)

for img_name in os.listdir(os.path.join(ruta_entrada, imagenes)):
  if not img_name.endswith('.png'):
    continue
  print(f"Procesando imagen: {img_name}")
  ruta_imagen = os.path.join(ruta_entrada, imagenes, img_name)
  ruta_prediccion = os.path.join(ruta_entrada, mascara_base, img_name)

  imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
  prediccion = cv2.imread(ruta_prediccion, cv2.IMREAD_GRAYSCALE)

  tipo = 'cervical' if 'cervical' in img_name.lower() else 'lumbar'
  segmentacion_limpia = procesar_mascara(prediccion, tipo,imagen)
  if segmentacion_limpia is not None:
    cv2.imwrite(os.path.join(ruta_salida, img_name), segmentacion_limpia)
    
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(prediccion, cmap='gray')
    plt.title("Segmentacion original")
    plt.subplot(1, 2, 2)
    plt.imshow(segmentacion_limpia, cmap='gray')
    plt.title("Segmentacion postprocesada")
    plt.tight_layout()

    # guardar figura en resultados_post_procesado
    ruta_resultados = os.path.join('resultados_post_procesado_julio', 'lumbar')
    os.makedirs(ruta_resultados, exist_ok=True)
    ruta_figura = os.path.join(ruta_resultados, f"{img_name}.png")
    plt.savefig(ruta_figura, bbox_inches='tight', dpi=300)

    plt.show()
  else:
    print(f"Segmentación nula para la imagen: {img_name}")
    plt.imshow(prediccion, cmap='gray')
    plt.title("Segmentacion original")
    plt.show()